# From basic LLM to A2A

1. **Basic LLM (Chatbot):** Input $\rightarrow$ LLM $\rightarrow$ Output.

   * *Example:* You ask "Write snake game code." ChatGPT returns the code. If the code is wrong, you must manually copy the error, paste it back, and tell it to fix it.

2. **Autonomous Agent (Tác nhân tự hành):** LLM + Tools + Memory.

   * *Example:* You assign a task. The agent thinks on its own, searches Google on its own, saves files on its own. However, it works alone (solo), making it easy to get "lost" or loop endlessly.

3. **Multi-Agent Collaboration:** A group of specialized Agents.

   * *Example:* A virtual software team including: PM (Manager), Coder (Programmer), Tester (Quality assurance). They talk to each other to create the final product.

In [19]:
%pip install -q google-generativeai colorama

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
from colorama import Fore, Style
import datetime

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)



/opt/homebrew/Caskroom/miniconda/base/envs/AI/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/homebrew/Caskroom/miniconda/base/envs/AI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_NAME = 'gemini-2.0-flash'

## Level 1: Basic Chatbot 

In [6]:
# --- Level 1 Code ---

def simple_chatbot(user_input):
    # Initialize model
    model = genai.GenerativeModel(MODEL_NAME)
    
    # Call generate_content function
    response = model.generate_content(user_input)
    return response.text


# Test
print(f"{Fore.GREEN}User:{Style.RESET_ALL} Explain Agent to Agent in short.")
answer = simple_chatbot("Explain Agent to Agent in short.")
print(f"{Fore.BLUE}Bot:{Style.RESET_ALL} {answer}")

User: Explain Agent to Agent in short.
Bot: Agent to Agent (A2A) refers to **direct communication and collaboration between two or more intelligent agents (software, robots, etc.) without direct human intervention.**  They exchange information, negotiate, and coordinate actions to achieve individual or collective goals.



In [8]:
print(f"{Fore.GREEN}User:{Style.RESET_ALL} What is the time now and what is the date today?")
answer = simple_chatbot("What is the time now and what is the date today?")
print(f"{Fore.BLUE}Bot:{Style.RESET_ALL} {answer}")

User: What is the time now and what is the date today?
Bot: I do not have access to real-time information, including the current time and date. To find out the current time and date, please check your device's clock or search on the internet.



## Level 2: Single Agent (Function Calling)

In [24]:
# --- Level 2 Code: Single Agent with Real-time Tool ---

# 1. Define Tool: Get current date and time
def get_current_date_and_time():
    now = datetime.datetime.now()
    # Format time beautifully - Use %B for full month name to avoid confusion
    time_str = now.strftime("%H:%M:%S - %B %d, %Y")
    
    # Print to show Agent is secretly calling this function
    print(f"{Fore.YELLOW}>>> [Tool] Looking at system clock...{Style.RESET_ALL}")
    return time_str

# 2. Add tool to list
tools_list = [get_current_date_and_time]

# 3. Initialize Agent
# Don't define complex system prompt, just give it the tool
model = genai.GenerativeModel(MODEL_NAME, tools=tools_list)
chat = model.start_chat(enable_automatic_function_calling=True)

# 4. Run test
user_query = "What time is it and what is the date today?"
print(f"{Fore.GREEN}User:{Style.RESET_ALL} {user_query}")

# Agent will automatically call get_current_time -> Get result -> Reply
response = chat.send_message(user_query)

print(f"{Fore.BLUE}Agent:{Style.RESET_ALL} {response.text}")

User: What time is it and what is the date today?
>>> [Tool] Looking at system clock...
Agent: Today is December 09, 2025 and the time is 14:54:34.



In [20]:
chat = model.start_chat(enable_automatic_function_calling=True)

user_query = "How many days until 1/1/2026?"
print(f"{Fore.GREEN}User:{Style.RESET_ALL} {user_query}")

response = chat.send_message(user_query)

print(f"{Fore.BLUE}Agent:{Style.RESET_ALL} {response.text}")

User: How many days until 1/1/2026?
Agent: I can get the current date and time, but I can't directly calculate the number of days until a future date. Would you like me to get the current date and time?



### With system prompt

In [25]:
sys_prompt = """
IMPORTANT: When you are asked to calculate the time left until a specific date, 
always use the current date and time to calculate the time left.
"""

# Use system prompt
model = genai.GenerativeModel(
    MODEL_NAME, 
    tools=tools_list, 
    system_instruction=sys_prompt
)
chat = model.start_chat(enable_automatic_function_calling=True)

user_query = "How many days until 1/1/2026?"
print(f"{Fore.GREEN}User:{Style.RESET_ALL} {user_query}")

response = chat.send_message(user_query)

print(f"{Fore.BLUE}Agent:{Style.RESET_ALL} {response.text}")

User: How many days until 1/1/2026?
>>> [Tool] Looking at system clock...
Agent: That means there are 23 days until 1/1/2026.



In [27]:
# --- Level 2 Challenge ---
user_request_game = "Write a Snake game in Python without using pygame. It should have a GUI and be playable."
print(f"{Fore.GREEN}User:{Style.RESET_ALL} {user_request_game}")

# Call Single Agent (already initialized above)
response = chat.send_message(user_request_game)

print(f"{Fore.BLUE}Agent:{Style.RESET_ALL} {response.text}")

print("\n" + "-"*50)

User: Write a Snake game in Python without using pygame. It should have a GUI and be playable.
Agent: I understand you're looking for a Snake game in Python with a GUI, but without using `pygame`. Creating a fully functional GUI-based game like Snake requires a library to handle the graphics, user input, and game loop. Since you've specified *not* to use `pygame`, here's what I can offer:

**Conceptual Outline (Important Limitations):**

1.  **GUI Library:** The most common alternative for a simple GUI in Python would be `tkinter`. However, `tkinter` is not well-suited for fast, animated games. It's more appropriate for creating applications with buttons, text fields, etc. If you're open to *any* GUI library besides `pygame`, then consider alternatives like `arcade` or `Pyglet`, though they might have dependencies you'd prefer to avoid.

2.  **Game Logic (The part I *can* provide):** I *can* give you the core game logic for Snake. This would include:
    *   Snake movement
    *   Food

In [4]:
import time
import random

class SnakeGame:
    def __init__(self, width=20, height=20):
        self.width = width
        self.height = height
        self.snake = [(width // 2, height // 2)]
        self.food = self.place_food()
        self.direction = 'right'
        self.score = 0
        self.game_over = False

    def place_food(self):
        while True:
            food = (random.randint(0, self.width - 1), random.randint(0, self.height - 1))
            if food not in self.snake:
                return food

    def move(self):
        head_x, head_y = self.snake[0]
        if self.direction == 'right':
            new_head = (head_x + 1, head_y)
        elif self.direction == 'left':
            new_head = (head_x - 1, head_y)
        elif self.direction == 'up':
            new_head = (head_x, head_y - 1)
        elif self.direction == 'down':
            new_head = (head_x, head_y + 1)

        if (new_head[0] < 0 or new_head[0] >= self.width or
            new_head[1] < 0 or new_head[1] >= self.height or
            new_head in self.snake):
            self.game_over = True
            return

        self.snake.insert(0, new_head)

        if new_head == self.food:
            self.score += 1
            self.food = self.place_food()
        else:
            self.snake.pop()

    def change_direction(self, direction):
        if direction == 'right' and self.direction != 'left':
            self.direction = 'right'
        elif direction == 'left' and self.direction != 'right':
            self.direction = 'left'
        elif direction == 'up' and self.direction != 'down':
            self.direction = 'up'
        elif direction == 'down' and self.direction != 'up':
            self.direction = 'down'
        elif direction == 'exit':
            self.game_over = True

    def draw(self):
        for y in range(self.height):
            for x in range(self.width):
                if (x, y) == self.food:
                    print('F', end='')
                elif (x, y) in self.snake:
                    print('S', end='')
                else:
                    print('.', end='')
            print()
        print(f"Score: {self.score}")

    def play(self):
        while not self.game_over:
            self.draw()
            direction = input("Nhập hướng đi (up, down, left, right) hoặc exit để thoát: ").lower()
            self.change_direction(direction)
            self.move()
            time.sleep(0.2)  # Adjust speed here
            print("\033[H\033[J", end="") # Clear screen

        print("Game Over!")
        print(f"Final Score: {self.score}")

if __name__ == "__main__":
    game = SnakeGame()
    game.play()

....................
.....F..............
....................
....................
....................
....................
....................
....................
....................
....................
..........S.........
....................
....................
....................
....................
....................
....................
....................
....................
....................
Score: 0
Game Over!
Final Score: 0


## Level 3: Multi-Agent System 


Mechanism of Multi-Agent Operation

The Multi-Agent model operates based on simulating human workflow:

*   **Role Playing:** Each Agent is assigned a "Persona."
    *   *Agent A:* "You are an SEO expert."
    *   *Agent B:* "You are a creative content writer."
    *   *Agent C:* "You are a strict editor."
*   **Conversation Pattern:**
    *   **Sequential:** A finishes and passes to B, B passes to C.
    *   **Hierarchical:** A "Boss Agent" distributes tasks to sub-agents and aggregates results.
    *   **Joint Chat:** Everyone discusses in a virtual chat group to find the best solution.
*   **Self-Correction:** This is the strongest point of A2A. If the Coder Agent writes incorrect code, the Tester Agent will run it, detect errors, and "scold" the Coder Agent (send back error logs). The Coder Agent will correct itself and resubmit to the Tester. This process repeats until the code runs correctly.

### Why is Multi-Agent more powerful than a single Chatbot?

1.  **Overcoming Context Window Limitations:** Instead of cramming all information into a single prompt, the work is broken down, and each agent only needs to handle its part.
2.  **Reducing Hallucination:** When one agent provides incorrect information, another agent (with a critical/checking role) can detect it and request correction. The "debate" among AIs helps increase accuracy.
3.  **Specialization:** An LLM carefully prompted to be an "SQL Expert" will write better queries than an LLM prompted to be a "General Assistant."
4.  **Complex Automation:** Multi-Agent can perform long sequences of tasks: *Market research $\rightarrow$ Report writing $\rightarrow$ PDF file creation $\rightarrow$ Email sending.*

## Sequential

#### Step 1: Define the Agent class (Gemini Wrapper)

In [28]:
class Agent:
    def __init__(self, name, system_instruction):
        self.name = name
        # Initialize a separate model for each Agent with its own "personality" (system instruction)
        self.model = genai.GenerativeModel(
            MODEL_NAME, 
            system_instruction=system_instruction
        )
        self.chat_session = self.model.start_chat(history=[])

    def reply(self, content):
        response = self.chat_session.send_message(content)
        return response.text

#### Step 2: Create Worker Agents

In [29]:
# Coder Agent
coder = Agent(
    name="Coder",
    system_instruction="You are a Senior Python Dev. Task: Write Python code to solve problems. Only return code, no explanations."
)

# Reviewer Agent
reviewer = Agent(
    name="Reviewer",
    system_instruction="You are a QA Engineer. Task: Review code, check for logical/security errors. If good, reply 'APPROVED'. If not, point out the errors."
)

#### Create Supervisor (JSON Output)

In [30]:
def supervisor_node(history_summary):
    system_instruction = """
    You are the Project Manager. You have employees: ['Coder', 'Reviewer'].
    Process:
    1. Request code -> Assign 'Coder'.
    2. Coder finishes -> Assign 'Reviewer'.
    3. Reviewer 'APPROVED' -> Respond 'FINISH'.
    4. Reviewer finds errors -> Reassign 'Coder' for fixes.
    
    OUTPUT JSON FORMAT: {"next_agent": "Agent_Name_Or_FINISH", "instruction": "Instructions for that agent"}
    """
    
    # Configure JSON response
    model = genai.GenerativeModel(
        MODEL_NAME, 
        system_instruction=system_instruction,
        generation_config={"response_mime_type": "application/json"}
    )
    
    response = model.generate_content(f"Current project status: {history_summary}")
    return json.loads(response.text)

#### Run A2A Loop

In [31]:
def run_a2a_gemini(user_prompt):
    print(f"{Fore.GREEN}User Request:{Style.RESET_ALL} {user_prompt}\n")
    
    # Context for Supervisor to understand the situation
    current_context = f"User request: {user_prompt}"
    
    # Loop for a maximum of 6 steps
    for i in range(6):
        # 1. Supervisor makes a decision
        decision = supervisor_node(current_context)
        next_agent_name = decision.get("next_agent")
        instruction = decision.get("instruction")
        
        print(f"{Fore.MAGENTA}--- Supervisor: Assigning to {next_agent_name} (Note: {instruction}) ---{Style.RESET_ALL}")
        
        if next_agent_name == "FINISH":
            print(f"\n{Fore.GREEN}Process complete! Output approved.{Style.RESET_ALL}")
            break
            
        # 2. Select Agent
        worker = None
        if next_agent_name == "Coder": worker = coder
        elif next_agent_name == "Reviewer": worker = reviewer
            
        # 3. Agent works
        if worker:
            # Agent receives context from previous steps (simplified by string concatenation)
            response = worker.reply(f"{instruction}\nPrevious information: {current_context}")
            
            print(f"{Fore.CYAN}[{next_agent_name}]:{Style.RESET_ALL} {response}\n")
            
            # 4. Update context for the next iteration
            current_context = f"Result from {next_agent_name}: {response}"

# --- EXECUTION ---
run_a2a_gemini("Write a Snake game in Python without using pygame. It should have a GUI and be playable.")

User Request: Write a Snake game in Python without using pygame. It should have a GUI and be playable.

--- Supervisor: Assigning to Coder (Note: Write a Snake game in Python without using pygame. The game should have a graphical user interface (GUI) and be fully playable. Consider using libraries like Tkinter or similar for the GUI. Focus on creating a well-structured, functional, and engaging game.) ---
[Coder]: ```python
import tkinter as tk
import random

class SnakeGame:
    def __init__(self, master):
        self.master = master
        master.title("Snake Game")

        self.width = 600
        self.height = 400
        self.cell_size = 20
        self.snake_color = "green"
        self.food_color = "red"
        self.background_color = "black"
        self.delay = 100 

        self.canvas = tk.Canvas(master, width=self.width, height=self.height, bg=self.background_color, highlightthickness=0)
        self.canvas.pack()

        self.snake = [(100, 100), (80, 100), (60, 100)]

In [31]:
import tkinter as tk
import random

# Constants
WIDTH = 600
HEIGHT = 600
CELL_SIZE = 20
SNAKE_COLOR = "green"
FOOD_COLOR = "red"
BACKGROUND_COLOR = "black"
TEXT_COLOR = "white"
FONT = ("Arial", 16)
GAME_SPEED = 100  # milliseconds

class SnakeGame:
    def __init__(self, master):
        self.master = master
        master.title("Snake Game")

        self.width = WIDTH
        self.height = HEIGHT
        self.cell_size = CELL_SIZE
        self.board_width = self.width // self.cell_size
        self.board_height = self.height // self.cell_size

        self.canvas = tk.Canvas(master, width=self.width, height=self.height, bg=BACKGROUND_COLOR)
        self.canvas.pack()

        self.score_label = tk.Label(master, text="Score: 0", font=FONT)
        self.score_label.pack()

        self.direction_label = tk.Label(master, text="Direction: Right", font=FONT)
        self.direction_label.pack()

        self.bind_keys()
        self.initialize_game()
        self.start_game()

    def initialize_game(self):
        self.snake = [(self.board_width // 2, self.board_height // 2)]
        self.food = self.create_food()
        self.direction = "Right"
        self.score = 0
        self.update_score_label()
        self.update_direction_label()

    def bind_keys(self):
        self.master.bind("<Up>", lambda event: self.change_direction("Up"))
        self.master.bind("<Down>", lambda event: self.change_direction("Down"))
        self.master.bind("<Left>", lambda event: self.change_direction("Left"))
        self.master.bind("<Right>", lambda event: self.change_direction("Right"))
        self.master.bind("q", lambda event: self.master.destroy()) # Press 'q' to quit

    def start_game(self):
        self.move_snake()

    def create_food(self):
        while True:
            x = random.randint(0, self.board_width - 1)
            y = random.randint(0, self.board_height - 1)
            if (x, y) not in self.snake:
                return (x, y)

    def move_snake(self):
        head_x, head_y = self.snake[0]

        if self.direction == "Right":
            new_head = (head_x + 1, head_y)
        elif self.direction == "Left":
            new_head = (head_x - 1, head_y)
        elif self.direction == "Up":
            new_head = (head_x, head_y - 1)
        elif self.direction == "Down":
            new_head = (head_x, head_y + 1)

        # Check for collision with walls or self
        if (
            new_head[0] < 0
            or new_head[0] >= self.board_width
            or new_head[1] < 0
            or new_head[1] >= self.board_height
            or new_head in self.snake
        ):
            self.game_over()
            return

        self.snake.insert(0, new_head)

        # Check if snake ate the food
        if new_head == self.food:
            self.score += 10
            self.food = self.create_food()
            self.update_score_label()
        else:
            self.snake.pop()

        self.draw()
        self.master.after(GAME_SPEED, self.move_snake)  # Adjust speed here (milliseconds)

    def change_direction(self, new_direction):
        # Prevent snake from reversing direction immediately
        if len(self.snake) > 1:
            if (new_direction == "Right" and self.direction == "Left") or \
               (new_direction == "Left" and self.direction == "Right") or \
               (new_direction == "Up" and self.direction == "Down") or \
               (new_direction == "Down" and self.direction == "Up"):
                return  # Ignore invalid direction
        self.direction = new_direction
        self.update_direction_label()

    def draw(self):
        self.canvas.delete("all")  # Clear the canvas

        # Draw snake
        for x, y in self.snake:
            x1 = x * self.cell_size
            y1 = y * self.cell_size
            x2 = x1 + self.cell_size
            y2 = y1 + self.cell_size
            self.canvas.create_rectangle(x1, y1, x2, y2, fill=SNAKE_COLOR)

        # Draw food
        x, y = self.food
        x1 = x * self.cell_size
        y1 = y * self.cell_size
        x2 = x1 + self.cell_size
        y2 = y1 + self.cell_size
        self.canvas.create_oval(x1, y1, x2, y2, fill=FOOD_COLOR)

    def game_over(self):
        self.canvas.delete("all")
        self.canvas.create_text(
            self.width / 2,
            self.height / 2,
            text=f"Game Over! Score: {self.score}",
            font=FONT,
            fill=TEXT_COLOR,
        )
        restart_button = tk.Button(self.master, text="Restart", command=self.restart_game)
        restart_button.pack()

    def restart_game(self):
        for widget in self.master.winfo_children():
            widget.destroy()
        self.__init__(self.master) #reinitialize the game

    def update_score_label(self):
        self.score_label.config(text=f"Score: {self.score}")

    def update_direction_label(self):
        self.direction_label.config(text=f"Direction: {self.direction}")

root = tk.Tk()
game = SnakeGame(root)
root.mainloop()

## Parallel Collaboration

In [32]:
# Unlike Sequential, this model allows Agents to work in parallel
# to solve a problem from their unique perspectives, after which a Leader synthesizes the results.

print(f"{Fore.CYAN}Parallel Collaboration (Council of Experts) ---{Style.RESET_ALL}")

# 1. Define Experts with specialized perspectives
marketer = Agent(
    name="Marketer",
    system_instruction="You are the CMO. Evaluate the idea based on market potential, viral capability, and revenue. Be strict and realistic."
)

tech_lead = Agent(
    name="TechLead",
    system_instruction="You are the CTO. Evaluate technical feasibility, coding difficulty, and necessary technologies (AI, App, Cloud)."
)

lawyer = Agent(
    name="Lawyer",
    system_instruction="You are a Lawyer. Warn about legal risks, copyright, data privacy, and lawsuits."
)

# 2. Define the Boss (Leader) who makes the final decision
boss = Agent(
    name="Boss",
    system_instruction="You are the CEO of Shark Tank. Read reports from the Marketer, TechLead, and Lawyer. Decide: INVEST or REJECT. Explain the reason briefly."
)

# 3. The Challenge: A 'Crazy' Startup Idea
startup_idea = "The 'Cat Tinder' app: AI translates cat meows to find partners for cats or owners with compatible personalities. Monthly fee: $50."
print(f"{Fore.GREEN}Startup Idea:{Style.RESET_ALL} {startup_idea}\n")

# 4. RUN IN PARALLEL (Fan-out)
# Instead of waiting for one to finish before the next, we send requests to all three simultaneously.
print(f"{Fore.YELLOW}>>> Consulting the council of experts...{Style.RESET_ALL}")

# These agents operate independently, unaware of each other's existence
report_marketing = marketer.reply(startup_idea)
report_tech = tech_lead.reply(startup_idea)
report_legal = lawyer.reply(startup_idea)

# Print individual results
print(f"\n{Fore.BLUE}[Marketer Report]:{Style.RESET_ALL}\n{report_marketing}\n")
print(f"{Fore.MAGENTA}[TechLead Report]:{Style.RESET_ALL}\n{report_tech}\n")
print(f"{Fore.RED}[Lawyer Report]:{Style.RESET_ALL}\n{report_legal}\n")

# 5. SYNTHESIS (Fan-in)
# Gather all reports and send them to the Boss for the final decision
final_report_prompt = f"""
Summary evaluation of the project "{startup_idea}":

--- MARKETING REPORT ---
{report_marketing}

--- TECH REPORT ---
{report_tech}

--- LEGAL REPORT ---
{report_legal}

Based on the 3 reports above, make the final judgment as CEO!
"""

print(f"{Fore.YELLOW}>>> CEO is contemplating...{Style.RESET_ALL}")
final_decision = boss.reply(final_report_prompt)

print(f"\n{Fore.GREEN}[CEO FINAL DECISION]:{Style.RESET_ALL}\n{final_decision}")

Parallel Collaboration (Council of Experts) ---
Startup Idea: The 'Cat Tinder' app: AI translates cat meows to find partners for cats or owners with compatible personalities. Monthly fee: $50.

>>> Consulting the council of experts...

[Marketer Report]:
Alright team, let's talk about "Cat Tinder," the app that purports to match cats (or their owners) based on meow translation and personality compatibility. As CMO, it's my job to cut through the fluff and assess the true potential. Here's my brutally honest take:

**Market Potential: Low to Moderate, Heavily Niche**

*   **Pro:** The pet industry is HUGE. People love their pets and spend a fortune on them. Anything that taps into this emotional connection has a baseline potential. Cat ownership is significant.
*   **Con:**
    *   **Niche within a Niche:** We're not just targeting pet owners; we're targeting *cat owners* who are *actively seeking* partners for their cats or themselves based on their cat's perceived preferences. This is

# Human-in-the-loop

In [33]:
# --- A2A SCENARIO: CREATIVE MARKETING TEAM (HUMAN-IN-THE-LOOP) ---
# This example creates a Marketing team to write social media content
# And includes a HUMAN FINAL APPROVAL step

import json
from colorama import Fore, Style
import google.generativeai as genai

# 1. Define Worker Agents
# Copywriter: Creative, imaginative
copywriter = Agent(
    name="Copywriter",
    system_instruction="""You are a creative, youthful Copywriter specializing in trendy Facebook/TikTok ad content.
    Task: Write short, engaging posts with many emojis based on the request. Only return the post content."""
)

# Editor: Strict, meticulous
editor = Agent(
    name="Editor",
    system_instruction="""You are a very strict Editor.
    Task: Review the Copywriter's post.
    1. If the post is poor, has spelling errors, or isn't engaging enough: Gently reprimand and ask for revisions (with suggestions).
    2. If the post is Perfect: Reply only with the keyword 'PUBLISH'.
    3. Only conclude when it's absolutely perfect."""
)

# 2. Define Supervisor for the Marketing team
def supervisor_marketing(history_summary):
    system_instruction = """
    You are the Marketing Manager (CMO). You manage two employees: ['Copywriter', 'Editor'].
    Workflow:
    1. Receive the task from the User -> Assign 'Copywriter' to draft.
    2. Copywriter finishes writing -> Assign 'Editor' to review.
    3. Editor responds 'PUBLISH' -> You (Supervisor) decide 'FINISH'.
    4. Editor OR Customer (Human) requests revisions -> Reassign to 'Copywriter' to revise according to their feedback.
    
    OUTPUT JSON FORMAT: {"next_agent": "Agent_Name_Or_FINISH", "instruction": "Specific instructions for the agent"}
    """
    
    # Configure to return JSON for easier logic processing
    model = genai.GenerativeModel(
        MODEL_NAME, 
        system_instruction=system_instruction,
        generation_config={"response_mime_type": "application/json"}
    )
    
    response = model.generate_content(f"Current work progress:\n{history_summary}")
    return json.loads(response.text)

# 3. Function to run the Marketing process (The Loop) - Version with a minimum KPI of 5 rounds
def run_marketing_team(topic):
    print(f"{Fore.GREEN}Customer Request:{Style.RESET_ALL} {topic}\n")
    print(f"{Fore.RED}--- STARTING MARKETING CAMPAIGN ---{Style.RESET_ALL}\n")
    
    # Summary of history
    history = f"Initial request: {topic}"
    
    # Set minimum loop KPI
    MIN_LOOPS = 5 
    
    for i in range(20): # Limit to 20 rounds for safety
        print(f"\n--- Round {i+1} ---")
        
        # Step 1: Supervisor decides who does what
        decision = supervisor_marketing(history)
        next_agent = decision.get("next_agent")
        instruction = decision.get("instruction")
        
        # --- NEW LOGIC: CHECK STOP CONDITION AND HUMAN-IN-THE-LOOP ---
        if next_agent == "FINISH":
            # Case 1: Not enough 5 rounds yet -> Force to continue
            if i < MIN_LOOPS:
                print(f"{Fore.MAGENTA}--- Supervisor wants to FINISH, but the system refuses (Only {i+1}/{MIN_LOOPS} rounds completed) ---{Style.RESET_ALL}")
                next_agent = "Copywriter"
                instruction = "The post is quite good, but I want you to create more unique variations or change the tone/mood. Don't stop too early."
                # Do not break, let the code continue below for the Copywriter to work
            
            # Case 2: Enough rounds completed -> Ask for Human approval
            else:
                print(f"\n{Fore.BLUE}>>> Team has completed (KPI {i+1} rounds). Waiting for Human approval...{Style.RESET_ALL}")
                print("Options: [ok] to Publish, or enter feedback to revise.")
                human_feedback = input("Human Review: ")
                
                if human_feedback.lower() == 'ok':
                    print(f"\n{Fore.GREEN}>>> CAMPAIGN SUCCESSFUL! POST APPROVED BY HUMAN.{Style.RESET_ALL}")
                    break
                else:
                    # Human rejects -> Feedback is added to history
                    print(f"{Fore.RED}>>> Human requested revision: {human_feedback}{Style.RESET_ALL}\n")
                    history += f"\n- CUSTOMER (HUMAN) REQUESTED: {human_feedback}. Revise immediately."
                    continue 
        
        # ----------------------------------------------------------------
        
        print(f"{Fore.MAGENTA}--- Supervisor: Assigning to {next_agent} (Note: {instruction}) ---{Style.RESET_ALL}")

        # Step 2: Route to the correct employee
        response_content = ""
        if next_agent == "Copywriter":
            response_content = copywriter.reply(instruction)
            print(f"{Fore.CYAN}[Copywriter]:{Style.RESET_ALL} {response_content}\n")
            history += f"\n- Copywriter wrote: {response_content}"
            
        elif next_agent == "Editor":
            response_content = editor.reply(f"Here is the latest post, please review:\n{history}")
            print(f"{Fore.YELLOW}[Editor]:{Style.RESET_ALL} {response_content}\n")
            history += f"\n- Editor commented: {response_content}"
            
    else:
        print(f"{Fore.RED}>>> Budget exhausted (too many loops)! Project paused.{Style.RESET_ALL}")

# --- TEST RUN ---
run_marketing_team("Write a post selling 'Smart office sleeping pillow' for programmers.")

Customer Request: Write a post selling 'Smart office sleeping pillow' for programmers.

--- STARTING MARKETING CAMPAIGN ---


--- Round 1 ---
--- Supervisor: Assigning to Copywriter (Note: Please draft a blog post selling 'Smart office sleeping pillow' specifically targeting programmers. Focus on the benefits of quick rest and improved productivity.) ---
[Copywriter]: 💻 Code got you crashing? 😴 Recharge with our Smart Office Sleeping Pillow! 🚀

✨ Power naps = Superpowers! ✨ Programmers, boost your productivity by unlocking the secret weapon of quick rests. 🧠

💤 This ain't your grandma's pillow! Ergonomic design + premium materials = maximum comfort in minimal time. 🏆

🔥 Benefits:
*   ⚡️ Instant energy boost
*   🎯 Laser focus unlocked
*   📈 Productivity through the roof!

Say goodbye to burnout and hello to brilliant code! 😎 Grab your Smart Office Sleeping Pillow NOW! 👇 Limited stock! ⏳ #programmer #codinglife #sleep #productivity #office #tech #smartpillow #naps #rest #energy #focus


